In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import pyvinecopulib as pv
import scipy
import seaborn as sns
from scipy.stats import nct, laplace_asymmetric ,genhyperbolic, laplace , t
import pandas_ta as ta
import matplotlib.pyplot as plt

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.processing import HistoricalPrice, RandomVariable
# ['MSN', 'VNM', 'LIX', 'DBD', 'DHG','ABT', 'OPC', 'TRA'] 

In [36]:
hp = HistoricalPrice()
asset = 'VNM'

file = f"../data/{asset}_historical_price.csv"
order = f"../data/{asset}_historical_order.csv"
df = hp.read(file)
df = df.loc[:,[i for i in df.columns if 'order' not in i]]
order = hp.get_order_info(order)

# order = order.replace(0, np.nan).interpolate()
# order = order.replace(np.inf, np.nan).interpolate()
df = df.join(order).dropna()
df.index =  pd.to_datetime(df.index)
df['returns'] = df['close']/df['close'].shift(5)-1
df = df.dropna()
df.head()

,adj_close,close,open,high,low,volume,SoLenhMua,KLDatMua,KLTB1LenhMua,SoLenhDatBan,KLDatBan,KLTB1LenhBan,ChenhLechKL,returns
2023-07-10,72.10,72.1,71.4,72.7,71.2,6201102,3840.0,11730600.0,3054.84,7422.0,11345600.0,1529.0,385000.0,0.032951
2023-07-10,69.78,72.1,71.4,72.7,71.2,6201102,3840.0,11730600.0,3054.84,7422.0,11345600.0,1529.0,385000.0,0.022695
2023-07-11,72.80,72.8,72.1,73.4,71.7,6235300,4198.0,10214100.0,2433.09,6575.0,10735300.0,1633.0,-521200.0,0.032624
2023-07-11,70.46,72.8,72.1,73.4,71.7,6235300,4198.0,10214100.0,2433.09,6575.0,10735300.0,1633.0,-521200.0,0.026798
2023-07-12,73.50,73.5,72.9,73.5,72.6,4195800,3430.0,7251000.0,2113.99,4856.0,8377500.0,1725.0,-1126500.0,0.036671


In [37]:
# List of all indicators
df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [38]:
help(ta.wma)

Help on function wma in module pandas_ta.overlap.wma:

wma(close, length=None, asc=None, talib=None, offset=None, **kwargs)
    Weighted Moving Average (WMA)
    
    The Weighted Moving Average where the weights are linearly increasing and
    the most recent data has the heaviest weight.
    
    Sources:
        https://en.wikipedia.org/wiki/Moving_average#Weighted_moving_average
    
    Calculation:
        Default Inputs:
            length=10, asc=True
        total_weight = 0.5 * length * (length + 1)
        weights_ = [1, 2, ..., length + 1]  # Ascending
        weights = weights if asc else weights[::-1]
    
        def linear_weights(w):
            def _compute(x):
                return (w * x).sum() / total_weight
            return _compute
    
        WMA = close.rolling(length)_.apply(linear_weights(weights), raw=True)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 10
        asc (bool): Recent values weigh m

In [39]:
ALPHA = [0,0.05,0.1,0.4,0.6,0.9,0.95,1]

var = RandomVariable(df['returns'],laplace_asymmetric)
simulated_data = var.simulate(100000)
quantiles = var.get_quantiles(simulated_data)
quantiles
# var.get_quantiles(simulated_data)

{0: -0.3063040771536015,
 0.05: -0.057321904624276825,
 0.1: -0.03919801801050943,
 0.4: -0.0038387390521298,
 0.6: 0.0066354902206502975,
 0.9: 0.035757142129837126,
 0.95: 0.050413377846587226,
 1: 0.24064290409057465}

In [40]:
import itertools

def quantile_ranges(ALPHA):
    ranges = []
    for i in range(len(ALPHA)-1):
        t = tuple([ALPHA[i],ALPHA[i+1]])
        ranges.append(t)
    return ranges
q_ranges = quantile_ranges(ALPHA)
q_ranges

[(0, 0.05),
 (0.05, 0.1),
 (0.1, 0.4),
 (0.4, 0.6),
 (0.6, 0.9),
 (0.9, 0.95),
 (0.95, 1)]

In [41]:
quantile_range_returns = {}
for lower,upper in q_ranges:
    subset = simulated_data[(simulated_data<=quantiles[upper])&(simulated_data>quantiles[lower])]
    quantile_range_returns[(lower,upper)] = subset.sum()/len(subset)
quantile_range_returns    

{(0, 0.05): -0.08299401454011303,
 (0.05, 0.1): -0.04708557867078514,
 (0.1, 0.4): -0.017581102024645467,
 (0.4, 0.6): 0.0016703742615681453,
 (0.6, 0.9): 0.01799302959407292,
 (0.9, 0.95): 0.042200148798322526,
 (0.95, 1): 0.07134695596418311}

In [42]:
cols = ["("+ ",".join([str(x) for x in r]) + "]" for r in q_ranges]
tree_3_index = list(itertools.product(cols,cols))
index_3 = pd.MultiIndex.from_tuples(tree_3_index, names=["t-2", "t-1"])
index_3

MultiIndex([(  '(0,0.05]',   '(0,0.05]'),
            (  '(0,0.05]', '(0.05,0.1]'),
            (  '(0,0.05]',  '(0.1,0.4]'),
            (  '(0,0.05]',  '(0.4,0.6]'),
            (  '(0,0.05]',  '(0.6,0.9]'),
            (  '(0,0.05]', '(0.9,0.95]'),
            (  '(0,0.05]',   '(0.95,1]'),
            ('(0.05,0.1]',   '(0,0.05]'),
            ('(0.05,0.1]', '(0.05,0.1]'),
            ('(0.05,0.1]',  '(0.1,0.4]'),
            ('(0.05,0.1]',  '(0.4,0.6]'),
            ('(0.05,0.1]',  '(0.6,0.9]'),
            ('(0.05,0.1]', '(0.9,0.95]'),
            ('(0.05,0.1]',   '(0.95,1]'),
            ( '(0.1,0.4]',   '(0,0.05]'),
            ( '(0.1,0.4]', '(0.05,0.1]'),
            ( '(0.1,0.4]',  '(0.1,0.4]'),
            ( '(0.1,0.4]',  '(0.4,0.6]'),
            ( '(0.1,0.4]',  '(0.6,0.9]'),
            ( '(0.1,0.4]', '(0.9,0.95]'),
            ( '(0.1,0.4]',   '(0.95,1]'),
            ( '(0.4,0.6]',   '(0,0.05]'),
            ( '(0.4,0.6]', '(0.05,0.1]'),
            ( '(0.4,0.6]',  '(0.1,

In [43]:
temp = pd.DataFrame(data = [[0],[0],[0],[1]])
temp[(temp.shift(1)==0)&(temp.shift(2)==0)]

,0
0,NaN
1,NaN
2,0.0
3,1.0


In [44]:

tree_depth_3 = pd.DataFrame(None,index=index_3,columns=index_3)
def get_transition_matrices(df: pd.DataFrame,quantiles: dict ,quantile_ranges: dict) -> None:
    for current_a_low,current_a_high in quantile_ranges:
        for l_a_low,l_a_high in quantile_ranges:
            for ll_a_low,ll_a_high in quantile_ranges:
                conds = df[(df.shift(1)>quantiles[l_a_low])&(df.shift(1)<=quantiles[l_a_high])&(df.shift(2)>quantiles[ll_a_low])&(df.shift(2)<=quantiles[ll_a_high])]
                events = conds[(conds<=quantiles[current_a_high])&(conds>quantiles[current_a_low])]
                if len(conds)>=4:
                    t2 = "("+ ",".join([str(ll_a_low),str(ll_a_high)]) + "]"
                    t1 = "("+ ",".join([str(l_a_low),str(l_a_high)]) + "]"
                    t = "("+ ",".join([str(current_a_low),str(current_a_high)]) + "]"
                    prob = len(events)/len(conds)
                    tree_depth_3.loc[(t2,t1),(t1,t)] = prob

    # #### NOT THE STANDARD TRANSITION MATRIX where each row adds to 1
# #### becauses the output state are quantile-ranges that overlap
get_transition_matrices(df['returns'],quantiles,q_ranges)
tree_depth_3 = tree_depth_3.dropna(how='all',axis=1).dropna(how='all',axis=0)
tree_depth_3

t-2                   (0,0.05]                                           \
t-1                   (0,0.05] (0.05,0.1] (0.1,0.4] (0.4,0.6] (0.6,0.9]   
t-2        t-1                                                            
(0,0.05]   (0,0.05]       0.75        0.0      0.25       0.0       0.0   
(0.1,0.4]  (0.1,0.4]       NaN        NaN       NaN       NaN       NaN   
           (0.4,0.6]       NaN        NaN       NaN       NaN       NaN   
(0.4,0.6]  (0.1,0.4]       NaN        NaN       NaN       NaN       NaN   
           (0.4,0.6]       NaN        NaN       NaN       NaN       NaN   
(0.6,0.9]  (0.4,0.6]       NaN        NaN       NaN       NaN       NaN   
           (0.6,0.9]       NaN        NaN       NaN       NaN       NaN   
           (0.9,0.95]      NaN        NaN       NaN       NaN       NaN   
(0.9,0.95] (0.6,0.9]       NaN        NaN       NaN       NaN       NaN   

t-2                                       (0.1,0.4]                       ...  \
t-1                   (0.9,0.95] (0.95,1]  (0,0.05] (0.05,0.1] (0.1,0.4]  ...   
t-2        t-1                                                            ...   
(0,0.05]   (0,0.05]          0.0      0.0       NaN        NaN       NaN  ...   
(0.1,0.4]  (0.1,0.4]         NaN      NaN      0.04       0.08      0.68  ...   
           (0.4,0.6]         NaN      NaN       NaN        NaN       NaN  ...   
(0.4,0.6]  (0.1,0.4]         NaN      NaN       0.0      0.125       0.5  ...   
           (0.4,0.6]         NaN      NaN       NaN        NaN       NaN  ...   
(0.6,0.9]  (0.4,0.6]         NaN      NaN       NaN        NaN       NaN  ...   
           (0.6,0.9]         NaN      NaN       NaN        NaN       NaN  ...   
           (0.9,0.95]        NaN      NaN       NaN        NaN       NaN  ...   
(0.9,0.95] (0.6,0.9]         NaN      NaN       NaN        NaN       NaN  ...   

t-2                   (0.6,0.9]                      (0.9,0.95]             \
t-1                   (0.6,0.9] (0.9,0.95]  (0.95,1]   (0,0.05] (0.05,0.1]   
t-2        t-1                                                               
(0,0.05]   (0,0.05]         NaN        NaN       NaN        NaN        NaN   
(0.1,0.4]  (0.1,0.4]        NaN        NaN       NaN        NaN        NaN   
           (0.4,0.6]        NaN        NaN       NaN        NaN        NaN   
(0.4,0.6]  (0.1,0.4]        NaN        NaN       NaN        NaN        NaN   
           (0.4,0.6]        NaN        NaN       NaN        NaN        NaN   
(0.6,0.9]  (0.4,0.6]        NaN        NaN       NaN        NaN        NaN   
           (0.6,0.9]   0.619048   0.095238  0.047619        NaN        NaN   
           (0.9,0.95]       NaN        NaN       NaN        0.0        0.0   
(0.9,0.95] (0.6,0.9]        0.5       0.25       0.0        NaN        NaN   

t-2                                                                      
t-1                   (0.1,0.4] (0.4,0.6] (0.6,0.9] (0.9,0.95] (0.95,1]  
t-2        t-1                                                           
(0,0.05]   (0,0.05]         NaN       NaN       NaN        NaN      NaN  
(0.1,0.4]  (0.1,0.4]        NaN       NaN       NaN        NaN      NaN  
           (0.4,0.6]        NaN       NaN       NaN        NaN      NaN  
(0.4,0.6]  (0.1,0.4]        NaN       NaN       NaN        NaN      NaN  
           (0.4,0.6]        NaN       NaN       NaN        NaN      NaN  
(0.6,0.9]  (0.4,0.6]        NaN       NaN       NaN        NaN      NaN  
           (0.6,0.9]        NaN       NaN       NaN        NaN      NaN  
           (0.9,0.95]       0.0       0.0      0.75       0.25      0.0  
(0.9,0.95] (0.6,0.9]        NaN       NaN       NaN        NaN      NaN  

[9 rows x 35 columns]

In [25]:
#### Need to determine what state (quantile range) the returns belong to 
states = list(set( [i[0] for i in tree_depth_3.index] + [i[1] for i in tree_depth_3.index]))
states

['(0.1,0.4]', '(0.6,0.9]', '(0.4,0.6]', '(0.95,1]']

In [41]:
def get_state(row):
    all_states = []
    r = row['returns']
    for s in states:
        lq,uq = s[1:-1].split(",")
        if r > quantiles[float(lq)] and r <= quantiles[float(uq)]:
            all_states.append(s)
    return all_states

df['state'] = df.apply(lambda x: get_state(x),axis=1)
df.head()


,adj_close,close,open,high,low,volume,SoLenhMua,KLDatMua,KLTB1LenhMua,SoLenhDatBan,KLDatBan,KLTB1LenhBan,ChenhLechKL,returns,state
2023-07-06,130.9,130.9,130.2,132.5,129.0,50300,186.0,92500.0,497.31,113.0,81500.0,721.0,11000.0,-0.012821,"[(0.1,0.4]]"
2023-07-07,134.9,134.9,130.8,135.6,130.5,48700,170.0,139100.0,818.24,241.0,94800.0,393.0,44300.0,0.030558,[]
2023-07-10,135.0,135.0,136.3,137.0,132.1,38000,270.0,113600.0,420.74,287.0,123400.0,430.0,-9800.0,0.000741,"[(0.6,0.9]]"
2023-07-11,134.0,134.0,135.2,135.4,133.7,25200,114.0,42400.0,371.93,128.0,68400.0,534.0,-26000.0,-0.007407,"[(0.1,0.4]]"
2023-07-12,132.5,132.5,134.0,134.8,132.0,15400,140.0,43300.0,309.29,98.0,100800.0,1029.0,-57500.0,-0.011194,"[(0.1,0.4]]"


In [42]:
def get_action(current_state: tuple, next_state: tuple):
    s_2 = current_state[0][1:-1].split(",")
    s_1 = current_state[1][1:-1].split(",")
    s = next_state[1][1:-1].split(",")
    if float(s_2[1]) <= 0.4 and float(s_1[1]) <= 0.4 and float(s[1]) <= 0.4:
        ### 3 days of negtive returns
        return "buy"
    elif float(s_2[1]) >= 0.6 and float(s_1[1]) >= 0.6 and float(s[1]) >= 0.6:
        ## 3 days of positive returns
        return "sell"
    elif float(s_2[1]) <= 0.1 and float(s_1[0]) >= 0.6 and float(s[1]) <= 0.4:
        ## large negative return, small positive return, small negative return
        return "buy"
    elif float(s_2[1]) >= 0.9 and float(s_1[1]) <= 0.05 and float(s[1]) <= 0.4:
        ## large positive return, large negative return, small negative return 
        return "buy"
    elif float(s_2[1]) <= 0.6 and float(s_1[1]) <= 0.4 and float(s[1]) <= 0.4:
        ## median return, small negative return, small negative return 
        return "buy"
    elif float(s_2[1]) >= 0.6 and float(s_1[1]) <= 0.1 and float(s[1]) <= 0.4:
        # small positive return, large negative return, small negative return 
        return "buy"

In [43]:
for i in range(0,len(df)-2):
    t_2 = df.iloc[i]['state']
    t_1 = df.iloc[i+1]['state'] 
    state = list(itertools.product(t_2,t_1))
    
    NEXT_STATE = None
    PROB = 0 
    CURRENT_STATE = None
    for s in state:
        if s in tree_depth_3.index:
            next_t = tree_depth_3.loc[s].dropna()
            max_prob = next_t.max()
            most_likely_next_t = next_t[next_t==max_prob]
            # print(s)
            next_state  = most_likely_next_t.index[0]
            if max_prob >= PROB:
                PROB = max_prob
                NEXT_STATE = next_state
                CURRENT_STATE = s
    if NEXT_STATE != None:
        ns = NEXT_STATE[1][1:-1].split(",")
        ns = tuple(float(i) for i in ns)
        # df.loc[df.index[i+2],'pred_close'] = (1+quantile_range_returns[ns])*df.loc[df.index[i+1],'close']
        # df.loc[df.index[i+2],'pred_returns'] = quantile_range_returns[ns]
        
        ### THE CLOSE IS KNOWN at i+1
        ### BUY AT OPEN ###
        ### 
        df.loc[df.index[i+2],'action'] = get_action(CURRENT_STATE,NEXT_STATE)

# df['action'] = df['action'].fillna('hold')
df.head(50)

,adj_close,close,open,high,low,volume,SoLenhMua,KLDatMua,KLTB1LenhMua,SoLenhDatBan,KLDatBan,KLTB1LenhBan,ChenhLechKL,returns,state,action
2023-07-06,130.9,130.9,130.2,132.5,129.0,50300,186.0,92500.0,497.31,113.0,81500.0,721.0,11000.0,-0.012821,"[(0.1,0.4]]",NaN
2023-07-07,134.9,134.9,130.8,135.6,130.5,48700,170.0,139100.0,818.24,241.0,94800.0,393.0,44300.0,0.030558,[],NaN
2023-07-10,135.0,135.0,136.3,137.0,132.1,38000,270.0,113600.0,420.74,287.0,123400.0,430.0,-9800.0,0.000741,"[(0.6,0.9]]",NaN
2023-07-11,134.0,134.0,135.2,135.4,133.7,25200,114.0,42400.0,371.93,128.0,68400.0,534.0,-26000.0,-0.007407,"[(0.1,0.4]]",NaN
2023-07-12,132.5,132.5,134.0,134.8,132.0,15400,140.0,43300.0,309.29,98.0,100800.0,1029.0,-57500.0,-0.011194,"[(0.1,0.4]]",NaN
2023-07-13,132.1,132.1,133.9,134.9,132.0,29900,138.0,54800.0,397.10,113.0,150700.0,1334.0,-95900.0,-0.003019,"[(0.4,0.6]]",NaN
2023-07-14,130.0,130.0,132.0,132.0,129.1,69600,282.0,106500.0,377.66,128.0,187000.0,1461.0,-80500.0,-0.015897,"[(0.1,0.4]]",NaN
2023-07-17,127.0,127.0,129.9,129.9,127.0,70800,295.0,129000.0,437.29,132.0,204700.0,1551.0,-75700.0,-0.023077,"[(0.1,0.4]]",NaN
2023-07-18,129.8,129.8,127.0,130.0,127.0,92900,294.0,181700.0,618.03,120.0,141500.0,1179.0,40200.0,0.022047,[],NaN
2023-07-19,138.8,138.8,129.8,138.8,129.8,219200,436.0,400800.0,919.27,367.0,287200.0,783.0,113600.0,0.069337,[],NaN


In [44]:
# class Backtest:
#     def __init__(self,backtest_df: pd.DataFrame,risk_free_pa: float,initial_cash: float):
#         self.risk_free_pd = (1+risk_free_pa)**(1/365)-1
#         self.cash = initial_cash
#         self.backtest_df = backtest_df
#         self.trade_size = 0.02
#         self.shares = 0
#         self.purchase_history = {}
    
#     def test(self):
#         shares = 0
#         day = 0
#         for i in range(0,len(self.backtest_df)):
#             open_ = df.iloc[i]['open']
#             close_  = df.iloc[i]['close']
#             action_ = df.iloc[i]['action'] 
#             trade_size = self.trade_size*total_capital 
            
#             if shares>0:

                
#             else:
                
#                 if action_ == 'buy':
#                     shares += trade_size/open_
#                     cash -= trade_size
#                     price_bought = open_ 

#             total_capital = cash + shares*close_
#             # print(i, "CAPITAL:",total_capital,"CASH:",cash,"SHARES:",shares,"action:",action_)
#     def buy(self):
        
#     def sell(self):
        
#     def condition_1(self):
#         if open_/price_bought - 1 >= self.risk_free_pd*1.5:
#         cash+=shares*open_
#         shares = 0
    
#     def condition_2(self):
#             ##### DOESNT EXCEED RISK FREE RATE, but price is expected to go down today, so sell open 
#         elif action_=='sell':
#             ### STOP LOSS HERE
#             cash+=shares*open_
#             shares = 0

In [45]:
### EACH TRADE USES 2% of total capital 
cash = 30000 ###( 30 million)
shares = 0
price_bought = 0
risk_free_return = (1+0.065)**(1/365)-1
risk_premium = 0.0005
print(risk_free_return)
total_capital = cash
import random

##### BUY OPEN, SELL 
for i in range(0,len(df)):
    open_ = df.iloc[i]['open']
    close_  = df.iloc[i]['close']
    action_ = df.iloc[i]['action'] 
    trade_size = 0.02*total_capital 

    # action_ = random.choice(['buy', 'hold','sell'])
    if shares>0:
        if open_/price_bought - 1 >= risk_free_return + risk_premium:
            # print("SELL HERE")
            cash+=shares*open_
            shares = 0
            # print(open_)
            # print(close_)
            action_ = 'sell'
        ##### DOESNT EXCEED RISK FREE RATE, but price is expected to go down today, so sell open 
        elif action_=='sell':
            ### STOP LOSS HERE
            cash+=shares*open_
            shares = 0
    else:
        if action_ == 'buy':
            shares += trade_size/open_
            cash -= trade_size
            price_bought = open_ 
            
    total_capital = cash + shares*close_
    # print(i, "CAPITAL:",total_capital,"CASH:",cash,"SHARES:",shares,"action:",action_)
    # print("\n")
    # if i ==5:
        # break
        
print(total_capital)
print(len(df))

0.00017254858112680793
30000.0
100


In [46]:
(total_capital/30000 - 1)**(365/len(df)) - 1

-1.0

In [ ]:
### MARKOV PROCESSS LEADS TO NEGATIVE RETURNS (WORSE THAN RANDOM BUY SELL HOLD). Cannot use MArkov strategy alone.